In [ ]:
!unzip /content/digit-recognizer.zip

Archive:  /content/digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd 
df_train=pd.read_csv("/content/train.csv")
df_test=pd.read_csv("/content/test.csv")

In [ ]:
import numpy as np
pixel_data=df_train.iloc[:,1:785] #from col 1 to 785 
label=df_train["label"]

number=np.array(pixel_data)
number=number.reshape(len(pixel_data),28,28,1)

In [ ]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(number, label, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [ ]:
x = X_train.reshape(-1, 1)



In [2]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import load_digits
import numpy as np

def calculate_entropy_per_cluster(data, labels, num_clusters):
    """
    Calculates the entropy for each cluster.

    Args:
        data: The data matrix.
        labels: The labels of the clusters.
        num_clusters: The number of clusters.

    Returns:
        A list of entropy values, one for each cluster.
    """
    mnist_data = load_digits()
    #X = digits.data / 255

    entropy_per_cluster = []
    for i in range(num_clusters):
        cluster_indices = np.where(labels == i)[0]
        cluster_counts = np.bincount(mnist_data.target[cluster_indices], minlength=10)
        cluster_probs = cluster_counts / len(cluster_indices)
        entropy = -np.sum(cluster_probs * np.log2(cluster_probs + 1e-10))
        entropy_per_cluster.append(entropy)
    return entropy_per_cluster

# Load the MNIST dataset and preprocess it
mnist_data = load_digits()
normalized_data = mnist_data.data / 255.

# Define the list of cluster sizes to try
cluster_sizes = [10, 100, 200, 500]

# Define the number of times to run the algorithm
num_runs = 10

# Loop over each cluster size and run the algorithm
for cluster_size in cluster_sizes:
    for run_num in range(num_runs):
        print(f"Run {run_num+1} of {num_runs} with cluster size={cluster_size}")
        
        # Perform agglomerative clustering to get initial clusters
        agg_clustering = AgglomerativeClustering(n_clusters=cluster_size)
        initial_labels = agg_clustering.fit_predict(normalized_data)

        # Perform CURE algorithm to refine clusters
        representative_points = np.zeros((cluster_size, normalized_data.shape[1]))
        for j in range(cluster_size):
            # Identify representative point for each cluster
            cluster_indices = np.where(initial_labels == j)[0]
            cluster_points = normalized_data[cluster_indices]
            centroid = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - centroid, axis=1)
            representative_index = cluster_indices[np.argmax(distances)]
            representative_points[j] = normalized_data[representative_index]

        shrunk_data = np.delete(normalized_data, representative_index, axis=0)
        shrunk_labels = agg_clustering.fit_predict(shrunk_data)

        # Calculate entropy for each cluster
        entropy_per_cluster = calculate_entropy_per_cluster(shrunk_data, shrunk_labels, cluster_size)
        print(f"Entropy per cluster: {entropy_per_cluster}")

        # Calculate total entropy for all clusters
        total_entropy = np.sum(entropy_per_cluster)

        # Print the results
        print(f"Total entropy: {total_entropy:.4f}")


Run 1 of 10 with cluster size=10
Entropy per cluster: [2.3267338172005143, 1.694360453358955, 1.6516392398060837, 1.7804713194425865, 1.9448983406549458, 1.439570153398266, 1.4031354612971698, 1.4133707493286796, 2.0726984872264267, 2.0455129678434103]
Total entropy: 17.7724
Run 2 of 10 with cluster size=10
Entropy per cluster: [2.3267338172005143, 1.694360453358955, 1.6516392398060837, 1.7804713194425865, 1.9448983406549458, 1.439570153398266, 1.4031354612971698, 1.4133707493286796, 2.0726984872264267, 2.0455129678434103]
Total entropy: 17.7724
Run 3 of 10 with cluster size=10
Entropy per cluster: [2.3267338172005143, 1.694360453358955, 1.6516392398060837, 1.7804713194425865, 1.9448983406549458, 1.439570153398266, 1.4031354612971698, 1.4133707493286796, 2.0726984872264267, 2.0455129678434103]
Total entropy: 17.7724
Run 4 of 10 with cluster size=10
Entropy per cluster: [2.3267338172005143, 1.694360453358955, 1.6516392398060837, 1.7804713194425865, 1.9448983406549458, 1.439570153398266,